In [37]:
import os
import pickle
import keras as K
import numpy as np
import tensorflow as tf
import jams
import builtins

builtins.tf = tf

In [ ]:
def load_model(model_spec_path, weights_path):
    # Load the serialized model spec
    with open(model_spec_path, "rb") as fd:
        model_spec = pickle.load(fd)
    # Reconstruct the model from the serialized spec
    model = K.utils.deserialize_keras_object(model_spec, safe_mode=False)
    # Load the weights from the saved file
    model.load_weights(weights_path)
    return model

In [ ]:
output_path = "../working/chords/model_deep"  # set your output directory
split = 0  # or whichever split you're loading

model_spec_path = os.path.join(
    output_path, "fold{:02d}_model.pkl".format(split)
)
weights_path = os.path.join(
    output_path, "fold{:02d}_weights.keras".format(split)
)

model = load_model(model_spec_path, weights_path)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ cqt_mag (InputLayer)            │ (None, None, 216, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, None, 216, 1)   │             4 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, None, 216, 1)   │            26 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, None, 1, 36)    │         7,812 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, None, 36)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, None, 256)      │       127,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, None, 256)      │       296,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ chord_tag (TimeDistributed)     │ (None, None, 170)      │        43,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,426,402 (5.44 MB)

 Trainable params: 475,466 (1.81 MB)

 Non-trainable params: 2 (8.00 B)

 Optimizer params: 950,934 (3.63 MB)

In [ ]:
def rename_slashes_in_op_fields(op):
    """
    In-place rename of all slash-laden keys inside op.fields
    so that the new keys replace '/' with '_'.
    """
    # Must check that op.fields is actually a mutable dict
    if hasattr(op, "fields") and isinstance(op.fields, dict):
        new_dict = {}
        for old_key, old_val in op.fields.items():
            new_key = old_key.replace("/", "_")  # e.g. 'cqt/mag' -> 'cqt_mag'
            new_dict[new_key] = old_val
        # Now overwrite op.fields with slash-free keys
        op.fields = new_dict


def rename_slashes_in_pump_opmap(pump):
    """
    Go through pump.opmap, rename slash-based keys in each operator's fields.
    """
    for op_name, op in pump.opmap.items():
        # rename slashes in the operator's .fields
        rename_slashes_in_op_fields(op)

        # If you also need to rename the op_name itself if it had a slash,
        # do it here (though 'chord_struct', 'chord_tag', 'cqt' do not have slashes):
        # new_op_name = op_name.replace('/', '_')
        # if new_op_name != op_name:
        #     pump.opmap[new_op_name] = op


def rename_slashes_in_pump_ops_list(pump):
    """
    pump.ops is a list of the same operators, rename slash-based keys in each.
    """
    for op in pump.ops:
        rename_slashes_in_op_fields(op)

In [33]:
with open(
    "/Users/theo/School/2/MIR/final-project/FinalProjectMIR/working/chords/pump.pkl",
    "rb",
) as fd:
    pump = pickle.load(fd)

rename_slashes_in_pump_opmap(pump)
rename_slashes_in_pump_ops_list(pump)

In [38]:
with open(
    "/Users/theo/School/2/MIR/final-project/FinalProjectMIR/working/reference_annotations/multisonidos.jams"
) as fd:
    jam = jams.load(fd)

In [ ]:
input = np.load(
    "/Users/theo/School/2/MIR/final-project/FinalProjectMIR/working/chords/pump/multisonidos.npz"
)["cqt_mag"]

predictions = model.predict(input)[0]

ann = pump["chord_tag"].inverse(predictions)
print(ann)
results = jams.eval.chord(jam.annotations["chord", 0], ann)

print(results)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
{
  "annotation_metadata": {
    "curator": {
      "name": "",
      "email": ""
    },
    "annotator": {},
    "version": "",
    "corpus": "",
    "annotation_tools": "",
    "annotation_rules": "",
    "validation": "",
    "data_source": ""
  },
  "namespace": "chord",
  "data": [
    {
      "time": 0.0,
      "duration": 0.09287981859410431,
      "value": "G:maj",
      "confidence": 0.04014365375041962
    },
    {
      "time": 0.09287981859410431,
      "duration": 43.003356009070295,
      "value": "A#:7",
      "confidence": 0.34787335991859436
    }
  ],
  "sandbox": {},
  "time": 0,
  "duration": null
}
OrderedDict([('thirds', 0.0), ('thirds_inv', 0.0), ('triads', 0.0), ('triads_inv', 0.0), ('tetrads', 0.0), ('tetrads_inv', 0.0), ('root', 0.0), ('mirex', 0.0), ('majmin', 0.0), ('majmin_inv', 0.0), ('sevenths', 0.0), ('sevenths_inv', 0.0), ('underseg', 0.25464399092970513), ('overseg', 0.9953560090702948), ('seg', 0.25464399092970513